<a href="https://www.kaggle.com/code/tolgaturan28/fish-classifacition?scriptVersionId=202727328" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Derin Öğrenme Modeli Eğitimi

## Giriş
* Gerekli kütüphanelerin eklenmesi ve Gpu kullanımı

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf  # Import TensorFlow as 'tf'
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam  # Correct non-breaking space

warnings.filterwarnings('ignore')

# Check if GPUs are available
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

## Balıkların Veri Seti 
* Veri setinin bulunduğu konumu bir değişkene atanması

In [ ]:
directory = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

classes = [i for i in os.listdir(directory) if '.' not in i]
classes

## Veri Yükleme ve Etiketleme
* Etiketler ve dosya yollarını saklamak için 2 liste tanımlanır

In [ ]:
label = []
path = []

for dir_name, _, filenames in os.walk(directory):
    for filename in filenames:
        if dir_name.split()[-1] != 'GT':
            # Sadece .png dosyalarını ekle
            if os.path.splitext(filename)[-1] == '.png':
                label.append(os.path.basename(dir_name))
                path.append(os.path.join(dir_name, filename))

data = pd.DataFrame({'path': path, 'label': label})
data['path'] = path
data['label'] = label
data

## Balık Türlerinin Dağılımı
* Balık Türlerinin sayısını gösterir

In [ ]:
data.label.value_counts()

## Görsel Örneği
* Balık türlerinden örnek görüntüler içeren kod parçası

In [ ]:
cn = 0

plt.figure(figsize = (20,20))
for unique_label in data['label'].unique():
    plt.subplot(3,3,cn+1)
    plt.imshow(plt.imread(data[data['label'] == unique_label].iloc[0,0]))
    plt.title(unique_label)
    plt.axis('off')
    cn += 1

## Etiketleri Sayısal Veriye Dönüştürme
* Balık türlerini sayısal verilere dönüştürülür ve DataFrame'deki etiketleri update eder

In [ ]:
label_to_int = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_int)

## PNG Dosyalarını Yükleme Fonksiyonu
* Pathlardeki PNG dosyalarını yükler ve boyutlandırır

In [ ]:
# Görüntüleri yükleme fonksiyonu
def load_images(image_paths, target_size=(64, 64)):
    images = [img_to_array(load_img(img, target_size=target_size)) / 255.0 for img in image_paths]
    return np.array(images)

In [ ]:
# Görüntüleri yükle
image_paths = data['path'].tolist()  # Görüntü dosya yollarını al
images = load_images(image_paths)

## Etiketleri Kategorik Sınıflandırma
* Etiketleri sınıflandırarak temsil etmek için kullanılır

In [ ]:
# Etiketleri kategorik hale getir
num_classes = len(data['label'].unique())  # Sınıf sayısını al
labels = to_categorical(data['label'], num_classes=num_classes)  # Kategorik etiketlere dönüştür

## Verileri Train Ve Validation Olarak Ayırma
* Verileri ve etiketleri train ve test olarak ayırma

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

## ANN Modeli Oluşturma
* Input Layer katmanı ekleme
* Flatten katmanı görüntüyü düzleştirir



In [ ]:
# ANN Modeli Oluşturma
model = Sequential()

# Düzleştirme katmanı
model.add(Flatten(input_shape=(64, 64, 3)))  # Görüntüleri düzleştir
model.add(Dense(512, activation='relu'))  # İlk gizli katman
model.add(Dropout(0.2))

## Gizli Katmanlar
* Yapay Sinir Ağı modeline gizli katmanlar ekleyerek daha doğru bir eğitim izleyebiliriz

In [ ]:
# Gizli katmanlar
model.add(Dense(256, activation='relu'))  # İlk gizli katman
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))   # İkinci gizli katman
model.add(Dropout(0.2))

## Output Layer- Çıktı Katmanı
* Yapay Sinir Ağının çıkış katmanını tanımlanır

In [ ]:
# Çıkış katmanı
model.add(Dense(num_classes, activation='softmax'))  # Çıkış katmanı (sınıf sayısı kadar nöron)

## MODELİ DERLEME
* Performans değerlendirmesi için kullanılır.

In [ ]:
# Modeli derleme
# Modeli Derleyelim (Compile The Model)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## MODELİ EĞİTME
* Yapay Sinir Ağı modelini eğitilir

In [ ]:
# Modeli eğitme
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

## Eğitimin Sonuçlarını Görselleştirme
* Burda loss eğrileri ve accuracy eğrileri çok yakın olmalıdır


In [ ]:
# Eğitimin sonuçlarını görselleştirme
# Eğitim ve doğrulama kaybı
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kaybı')
plt.ylabel('Kaybı')
plt.xlabel('Epoch')
plt.legend()
plt.show()
# Eğitim ve doğrulama doğruluğu
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.ylabel('Doğruluk')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## Model Evaluation (Model Değerlendirme)
* Modelin performansını ölçmek için yapılan işlemler.

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

## Proje Sonuçlandırılması
Bu proje kapsamında yapay sinir ağları (ANN) kullanarak görüntü sınıflandırma modeli geliştirmeyi öğrendik. Aşağıda, proje boyunca öğrendiğimiz ana fikirler özetlenmiştir:

1. Veri Ön İşleme ve Bölme
Veri setini üçe ayırma: Modeli daha iyi değerlendirmek için veriyi üçe ayırmayı öğrendik:
Eğitim seti: Modelin öğrenmesi için kullanıldı.
Doğrulama seti: Modelin eğitim sırasında performansını değerlendirmek için.
Test seti: Modelin genel başarısını gerçek hayattaki gibi test etmek için.
Bu sayede eğitim sürecinde aşırı öğrenmenin (overfitting) önüne geçmeyi ve daha sağlıklı sonuçlar elde etmeyi öğrendik.
2. Yapay Sinir Ağı (ANN) Mimarisi
Katmanlar ekleme: Bir ANN modelinin farklı katmanlardan oluştuğunu öğrendik:
Flatten katmanı: Görüntü verisini düzleştiren bir katman.
Gizli katmanlar (Dense): Görüntüleri sınıflandırmak için kullandığımız tam bağlı katmanlar.
Dropout katmanı: Aşırı öğrenmeyi azaltmak için düğümlerin bir kısmını rastgele devre dışı bırakarak düzenlemeye yardımcı olur.
Çıkış katmanı (softmax): Çok sınıflı sınıflandırma işlemi için kullanılır.
3. Model Eğitimi ve Değerlendirilmesi
Modelin derlenmesi (compile): Modeli uygun bir optimizasyon algoritması (Adam) ve kayıp fonksiyonu (categorical_crossentropy) ile derleyerek öğrenme sürecini başlattık.
Doğrulama seti kullanımı: Eğitim sırasında doğrulama seti ile modelin performansını izleyip, eğitim verilerine aşırı uyum göstermemesi için düzenli olarak değerlendirme yapmayı öğrendik.
Eğitim ve doğrulama kayıplarını izleme: Eğitim süreci boyunca modelin doğruluk ve kayıp grafiğini gözlemleyerek, modelin eğitim sürecinde nasıl bir gelişim gösterdiğini analiz edebildik.
4. Test Seti Üzerinde Modelin Değerlendirilmesi
Modeli eğittikten sonra test verileri üzerinde değerlendirme yaparak, modelin daha önce hiç görmediği verilerdeki başarısını ölçtük.
Genelleme: Modelin yeni verilere karşı ne kadar iyi genelleme yaptığını gözlemledik.
5. Model Performansının Görselleştirilmesi
Eğitim ve doğrulama süreçlerini grafiklerle görselleştirerek, modelin hangi noktada gelişim gösterdiğini ve hangi noktalarda iyileştirme gerektiğini anladık.
Bu proje ile görüntü işleme, sinir ağları kullanarak sınıflandırma modelleri oluşturma, veri setlerinin yönetimi ve model performansını değerlendirme gibi önemli makine öğrenimi ve derin öğrenme kavramlarını pekiştirmiş olduk.